# Working

## Plot annual velocity at point against time

In [ ]:
# EXPECTED RUNTIME: ~2 minutes

import xarray as xr
import pandas as pd
import re
import tqdm
import geopandas as gpd

import glob

GLACIERS = ["DJ", "UI"]

# import s3fs
# s3 = s3fs.S3FileSystem(anon=True)
# s3_path = "s3://its-live-data/velocity_mosaic/v2/annual/"
# s3_files = s3.ls(s3_path)
# pattern = re.compile(r"ITS_LIVE_velocity_120m_RGI05A_(\d{4})_v02\.nc")
# annual_mosaics = [f for f in s3_files if pattern.search(f)]
# annual_mosaics = sorted(annual_mosaics[1:])
# with s3.open(annual_mosaics[0], mode="rb") as f_handle:
#     with xr.open_dataset(f_handle, engine="h5netcdf") as ds:

files = glob.glob("../data/velocity/*.nc")
pattern = re.compile(r"ITS_LIVE_velocity_120m_RGI05A_(\d{4})_v02\.nc")
annual_mosaics = [f for f in files if pattern.search(f)]
annual_mosaics = sorted(annual_mosaics[1:])

centreline_lines = gpd.read_file("../data/misc/centreline_lines.geojson")
centreline_points = gpd.read_file("../data/misc/centreline_points.geojson")
centreline_points = centreline_points[
    (centreline_points["cd"] <= 20000) & (centreline_points["cd"] >= 0)
]
centreline_points = centreline_points.sort_values(by=["glacier_code", "cd"]).reset_index()

distances = [1000, 10000]
centreline_points = centreline_points[centreline_points["cd"].isin(distances)].reset_index()

for mosaic in annual_mosaics:
    print(f"Working on {mosaic}")

    with xr.open_dataset(mosaic, engine="h5netcdf") as ds:
        for idx, point in tqdm.tqdm(centreline_points.iterrows()):
            point_x, point_y = point.geometry.x, point.geometry.y

            box_half_size = 500
            box_x_min = point_x - box_half_size
            box_x_max = point_x + box_half_size
            box_y_min = point_y - box_half_size
            box_y_max = point_y + box_half_size

            box_subset = ds.where(
                (ds["x"] >= box_x_min)
                & (ds["x"] <= box_x_max)
                & (ds["y"] >= box_y_min)
                & (ds["y"] <= box_y_max),
                drop=True,
            )

            median_velocity = box_subset["v"].median().item()
            median_velocity_error = box_subset["v_error"].median().item()

            year = int(mosaic.split("_")[-2])
            centreline_points.loc[idx, f"{year}_v"] = median_velocity
            centreline_points.loc[idx, f"{year}_v_err"] = median_velocity_error

# centreline_points.to_file("../data/velocity/velocity_points_1985_2022.geojson", driver="GeoJSON")

In [88]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.dates import DateFormatter, YearLocator
from matplotlib.offsetbox import AnchoredText
from matplotlib.gridspec import GridSpec

GLACIERS = ["DJ", "UI"]
SETTINGS = {
    "DJ": {
        "colour": [(0.122, 0.467, 0.706, 1.0), (0.122, 0.467, 0.706, 0.25)],
        "label": "Daugaard-Jensen",
    },
    "UI": {
        "colour": [(1.0, 0.498, 0.055, 1.0), (1.0, 0.498, 0.055, 0.25)],
        "label": "Uunartit Islands",
    },
}

years = [int(i.split("_")[-2]) for i in annual_mosaics]
centreline_points = centreline_points[centreline_points["cd"] == 1000]

# Global plot settings
plt.rcParams["figure.dpi"] = 1000
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 18
plt.rcParams["axes.labelsize"] = 18
plt.rcParams["xtick.labelsize"] = 18
plt.rcParams["ytick.labelsize"] = 18

fig = plt.figure(figsize=(16, 6))
gs = GridSpec(nrows=1, ncols=2, width_ratios=[1, 2], height_ratios=[1])
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1:])
ax = [ax0, ax1]

# Plot data on each subplot
for glacier in GLACIERS:
    glacier_data = centreline_points[centreline_points["glacier_code"] == glacier]

    for i, period in enumerate([years[:26], years[25:]]):
        plot_years = [pd.to_datetime(f"{y}-01-01") for y in period]
        plot_v = [(float(glacier_data[f"{y}_v"].values) / 1000) for y in period]
        plot_v_err = [(float(glacier_data[f"{y}_v_err"].values) / 1000) for y in period]
        ax[i].errorbar(
            plot_years,
            plot_v,
            plot_v_err,
            label=SETTINGS[glacier]["label"],
            marker="o",
            linewidth=2.5,
            color=SETTINGS[glacier]["colour"][1],
            markersize=8,
            markeredgecolor=SETTINGS[glacier]["colour"][0],
            markerfacecolor=SETTINGS[glacier]["colour"][0],
            ecolor=SETTINGS[glacier]["colour"][1],
            elinewidth=5,
            zorder=10,
            clip_on=False,
        )

# Set y-axis range to fit data and adds label
ax[0].set_ylim(bottom=2, top=5.5)
ax[1].set_ylim(bottom=2, top=5.5)
ax[0].set_ylabel("Velocity [km y$^{-1}$]", labelpad=20)

# Set x-axis range to fit data
ax[0].set_xlim(pd.to_datetime("1985-01-01"), pd.to_datetime("2010-01-01"))
ax[1].set_xlim(pd.to_datetime("2010-01-01"), pd.to_datetime("2024-01-01"))

# Set x-axis tick frequency (major)
ax[0].xaxis.set_major_locator(YearLocator(base=5))
ax[0].xaxis.set_major_formatter(DateFormatter("%Y"))
ax[1].xaxis.set_major_formatter(DateFormatter("%Y"))

# Set x-axis tick frequency (minor)
ax[0].xaxis.set_minor_locator(YearLocator(1))
ax[1].xaxis.set_minor_locator(YearLocator(1))

# Rotate x-axis tick labels
ax[0].tick_params(axis="x", rotation=70)
ax[1].tick_params(axis="x", rotation=70)

# Disable y-axis tick labels
ax[1].tick_params(labelleft=False)

# Remove whitespace around data on plot
ax[0].margins(x=0)
ax[1].margins(x=0)

# Add plot labels and advance/retreat text
ax[0].add_artist(
    AnchoredText(
        "(c)",
        loc="upper left",
        borderpad=0.1,
        prop=dict(size=18, family="Times New Roman", weight="semibold", c="black"),
        frameon=False,
    )
)
ax[1].add_artist(
    AnchoredText(
        "(d)",
        loc="upper left",
        borderpad=0.1,
        prop=dict(size=18, family="Times New Roman", weight="semibold", c="black"),
        frameon=False,
    )
)

# Add gridlines
ax[0].grid(
    True,
    which="major",
    axis="both",
    c="gray",
    linestyle=":",
    linewidth=0.5,
    alpha=0.8,
    zorder=0,
)
ax[1].grid(
    True,
    which="both",
    axis="both",
    c="gray",
    linestyle=":",
    linewidth=0.5,
    alpha=0.8,
    zorder=0,
)

plt.tight_layout()
# plt.subplots_adjust(wspace=0.01)
# plt.savefig(
#     "../img/terminus_boxes_1985_2023_time-series.png",
#     dpi=fig.dpi,
#     bbox_inches="tight",
# )
plt.show()

C:\Users\olive\AppData\Local\Temp\ipykernel_44576\4105529732.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  plot_v = [(float(glacier_data[f"{y}_v"].values)/1000) for y in period]
C:\Users\olive\AppData\Local\Temp\ipykernel_44576\4105529732.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  plot_v_err = [(float(glacier_data[f"{y}_v_err"].values)/1000) for y in period]
C:\Users\olive\AppData\Local\Temp\ipykernel_44576\4105529732.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Dep

## Plot sub-annual velocity time series

## Plot variables as maps

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import holoviews as hv
from bokeh.models import HoverTool
import geopandas as gpd

# Define the coordinates for the bounding box
bbox_y_min = -1866831
bbox_x_min = -312651
bbox_y_max = -1799771
bbox_x_max = -260844

centreline_points = gpd.read_file("../data/misc/centreline_points.geojson")
centreline_lines = gpd.read_file("../data/misc/centreline_lines.geojson")
centreline_dj = centreline_lines[centreline_lines["glacier_code"] == "DJ"]

bbox_x_min, bbox_y_min, bbox_x_max, bbox_y_max = centreline_dj.bounds.values[0]

mosaic_path = "..\\data\\velocity\\ITS_LIVE_velocity_120m_RGI05A_2022_v02.nc"

with xr.open_dataset(mosaic_path, engine="h5netcdf") as ds:
    # Step 1: Determine the dataset's extent
    x_min = ds["x"].min().item()  # Get minimum 'x' coordinate
    x_max = ds["x"].max().item()  # Get maximum 'x' coordinate
    y_min = ds["y"].min().item()  # Get minimum 'y' coordinate
    y_max = ds["y"].max().item()  # Get maximum 'y' coordinate

    print(f"Dataset extent: x = [{x_min}, {x_max}], y = [{y_min}, {y_max}]")

    # Slice the dataset with the updated bounding box
    ds_subset = ds.where(
        (ds["x"] >= bbox_x_min)
        & (ds["x"] <= bbox_x_max)
        & (ds["y"] >= bbox_y_min)
        & (ds["y"] <= bbox_y_max),
        drop=True,
    )

    # Optionally clip dataset to land ice mask
    landice_mask = ds_subset["landice"] > 0
    ds_subset = ds_subset.where(landice_mask, float("nan"))

    # Optional: Plot the subsetted data
    ds_subset["v"].plot(cmap="viridis")
    plt.title("v")
    plt.show()

    ds_subset["v_error"].plot(cmap="viridis")
    plt.title("v_error")
    plt.show()

    ds_subset["landice"].plot(cmap="viridis")
    plt.title("landice")
    plt.show()

    ds_subset["floatingice"].plot(cmap="viridis")
    plt.title("floatingice")
    plt.show()

    ds_subset["count"].plot(cmap="viridis")
    plt.title("count")
    plt.show()

# Datacube access & sampling

In [ ]:
import json
import logging

import geopandas as gpd
import numpy as np
import pandas as pd
import s3fs as s3
import xarray as xr

logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


def get_datacube_instances(points):
    """Identifies the ITS_LIVE Zarr datacubes corresponding to sampling points.

    Parameters
    ----------
    points : geopandas.GeoDataFrame
        GeoDataFrame containing points along a glacier centreline.

    Returns
    -------
    datacubes : list of str
        List unique datacube URLs.
    points : geopandas.GeoDataFrame
        points with corresponding datacube URL appended.

    See also
    --------
    sample_datacubes
    """

    # Reads ITS_LIVE Zarr datacube catalog as GeoDataFrame
    s3fs = s3.S3FileSystem(anon=True)
    catalog_url = "s3://its-live-data/datacubes/catalog_v02.json"
    with s3fs.open(catalog_url, "r") as datacube_catalog:
        datacube_catalog = json.load(datacube_catalog)
        datacube_catalog = gpd.GeoDataFrame.from_features(datacube_catalog, crs=4326)

    # Intersects points with datacube outlines
    points = (
        points.to_crs(4326)
        .overlay(datacube_catalog[["geometry", "zarr_url"]], how="intersection")
        .set_index(["glacier", "cd"])
        .to_crs(3413)
    )

    # Amends URLs
    points["zarr_url"] = (
        points["zarr_url"].str.replace("http:", "s3:").str.replace(".s3.amazonaws.com", "")
    )

    # Creates list of unique datacubes (as their URLs)
    datacubes = points["zarr_url"].unique().tolist()
    logger.info(f"Number of unique datacubes: {len(datacubes)}")

    return datacubes, points


def sample_datacubes(datacubes, points, spatial_averaging=True):
    """Samples ITS_LIVE Zarr datacube at points.

    Parameters
    ----------
    datacubes : list of str
        List unique datacube URLs.
    points : geopandas.GeoDataFrame
        GeoDataFrame containing points along a glacier centreline.
    spatial_averaging : bool
        Flag to sample in a 3x3 window around each point [default=True].

    Returns
    -------
    datasets : list of xarray.Dataset
        Sampled datasets with the dimensions:
            - mid_date  ...
            - glacier   ...
            - cd        binned distance along centreline
            - k         int (0-8) each point in the 3x3 kernel

    See also
    --------
    get_datacube_instances
    """

    # Creates storage needed for spatial averaging, if necessary
    if spatial_averaging:
        points["windows"] = pd.Series(dtype=object)

    # Creates storage for sampled datasets
    datasets = []

    for dc_url in datacubes:
        print(points.to_string())

        # Gets index of points within current the datacube
        z_idx = points["zarr_url"] == dc_url

        with xr.open_dataset(dc_url, engine="zarr", storage_options={"anon": True}) as dc:
            if not spatial_averaging:
                # Gets point x and y coords
                point_x = points.loc[z_idx, "x"].to_xarray()
                point_y = points.loc[z_idx, "y"].to_xarray()
                mask = ~np.isnan(point_x)

                # Queries datacube at point
                dc = dc.sel(x=point_x, y=point_y, method="nearest").sortby("mid_date")

            else:
                # Creates additional points for 3x3 kernal about each index
                diffs_x = np.abs(points.loc[z_idx, "x"].values - dc["x"].values[:, np.newaxis])
                diffs_y = np.abs(points.loc[z_idx, "y"].values - dc["y"].values[:, np.newaxis])
                x_idx = diffs_x.argmin(axis=0)
                y_idx = diffs_y.argmin(axis=0)
                windows = []
                for x, y in list(zip(x_idx, y_idx)):
                    window = []
                    for y_ in [y + 1, y, y - 1]:
                        for x_ in [x - 1, x, x + 1]:
                            window.append((x_, y_))
                    windows.append(window)

                # Appends window indicies to points GeoDataFrame
                points.loc[z_idx, "windows"] = pd.Series(
                    windows, index=points.loc[z_idx].index
                )

                # Unpacks list of windows
                query = (
                    points.loc[z_idx, "windows"]
                    .apply(pd.Series)
                    .reset_index()
                    .melt(["glacier", "cd"])
                    .sort_values(by=["glacier", "cd"])
                    .rename(columns={"variable": "k", "value": "idx"})
                    .set_index(["glacier", "cd", "k"])
                )

                # Unpacks each window's tuple
                query = query["idx"].apply(pd.Series).rename(columns={0: "x_idx", 1: "y_idx"})

                # Removes invalid indicies caused when points cross a datacube boundary
                invalid = (
                    (query["y_idx"] >= len(dc["y"]))
                    | (query["y_idx"] < 0)
                    | (query["x_idx"] >= len(dc["x"]))
                    | (query["x_idx"] < 0)
                )
                query = query[~invalid]

                # Gets point x and y coords
                point_x = query["x_idx"].to_xarray()
                point_y = query["y_idx"].to_xarray()
                mask = ~np.isnan(point_x)

                # Handles NaN values created when dealing with centrelines of differing length
                point_x = xr.where(mask, point_x, 0).astype(int)
                point_y = xr.where(mask, point_y, 0).astype(int)

                # Queries datacube at point
                dc = dc.isel(x=point_x, y=point_y).sortby("mid_date")

            # Masks NaN values
            dc["x"] = xr.where(mask, dc["x"], np.nan)
            dc["y"] = xr.where(mask, dc["y"], np.nan)
            dc["mask"] = mask

            logger.info(f"Datacube dimensions: {dc.dims}")
            datasets.append(dc.chunk(chunks={"cd": 15, "glacier": 1}))

    logger.info(f"Number of datacubes loaded: {len(datasets)}")

    return datasets


def get_time_series(datasets, start_d, stop_d, distances, date_dt_max="30D"):
    """Gets velocity time series at points and resamples them to regular frequencies.

    Parameters
    ----------
    datasets : list of xarray.Dataset
        Sampled IT_LIVE data.
    start_d : str
        pandas timedelta string for filtering on mid_date.
    stop_d : str
        pandas timedelta string for filtering on mid_date.
    distances : list
        list of distances along centreline (cd) to include.
    date_dt_max : str
        pandas timedelta string specifying maximum date_dt [default=30].


    Returns
    -------
    v_time_series : pandas.DataFrame
        DataFrame of velocities that meet date, date_dt, and cd filter requirements.
            - glacier, cd, mid_date [with frequency = resample_frequency), v, season, year

    See also
    --------
    error_weighted_average
    get_seasonal_averages
    get_time_series
    v_uncertainty
    """

    filtered = []

    for ds in datasets:
        d_idx = ds["mid_date"].to_pandas().between(start_d, stop_d)
        date_dt_idx = ds["date_dt"] <= pd.Timedelta(date_dt_max)
        mid_date_idx = date_dt_idx & d_idx
        cd_idx = ds["cd"].isin(distances)

        if "k" not in list(ds.dims):  # then spatial averaging
            average_v, average_v_error = v_uncertainty(
                ds["vx"][mid_date_idx, :, cd_idx],
                ds["vy"][mid_date_idx, :, cd_idx],
                ds["vx_error"][mid_date_idx],
                ds["vy_error"][mid_date_idx],
            )
        else:
            average_vx = ds["vx"][mid_date_idx, :, cd_idx, :].mean(dim="k", skipna=True)
            average_vy = ds["vy"][mid_date_idx, :, cd_idx, :].mean(dim="k", skipna=True)
            average_v, average_v_error = v_uncertainty(
                average_vx,
                average_vy,
                ds["vx_error"][mid_date_idx],
                ds["vy_error"][mid_date_idx],
            )

        filtered.append(
            xr.merge(
                [
                    average_v.rename("v"),
                    average_v_error.rename("v_error"),
                    ds[
                        [
                            "acquisition_date_img1",
                            "acquisition_date_img2",
                            "date_dt",
                            "satellite_img1",
                            "mission_img1",
                        ]
                    ].sel(mid_date=mid_date_idx),
                ]
            )
        )

    print(filtered)

    # Concatenate datasets along the mid_date dimension
    concatenated = xr.concat(filtered, dim="mid_date", combine_attrs="override")

    # Convert to DataFrame and drop duplicates based on mid_date
    concatenated_df = concatenated.to_dataframe().reset_index()
    concatenated_df = concatenated_df.drop_duplicates(subset=["mid_date", "glacier", "cd"])

    # Filter out NaN values in 'v'
    v_time_series_df = concatenated_df[~concatenated_df["v"].isna()]

    # 28-day rolling median
    rolling_median = (
        v_time_series_df.set_index("mid_date")
        .groupby(["glacier", "cd"])["v"]
        .rolling("28D")
        .median()
        .rename("smoothed_v")
        .reset_index()
    )

    v_time_series_df = v_time_series_df.merge(
        rolling_median, on=["mid_date", "glacier", "cd"], how="left"
    )

    # Calculate intermediate steps for error-weighted averages
    v_time_series_df["v_over_sigma2"] = v_time_series_df["v"] / (
        v_time_series_df["v_error"] ** 2
    )
    v_time_series_df["one_over_sigma2"] = 1 / (v_time_series_df["v_error"] ** 2)

    return v_time_series_df